In [1]:
import numpy as np
import pandas as pd

from dataset_processing import split_dataset, get_starting_dataset, enrich_dataset, get_movie_map
from similarity import pearson_similarity
from predict import prediction_function
from group import alpha_hybrid_aggregation, update_alpha_mom_disagreements
from evaluate import user_satisfaction

from datetime import datetime

In [2]:
NUM_ITERATIONS = 5
MAX_NEIGHBORS = 75      # ~ 3*np.sqrt(num_users)
MAX_RECOMMENDATIONS = 10

In [3]:
movie_map = get_movie_map()

In [42]:
group = [437, 329, 73]

In [55]:
it_counter = 1
alpha = 0
df_user_sat = pd.DataFrame(index=group, columns=["It1", "It2", "It3", "It4", "It5"])
df_user_satO = pd.DataFrame(index=group, columns=["It1", "It2", "It3", "It4", "It5"])
group_disagreements = list()

In [56]:
while it_counter < 6:
    print(f"ITERATION: {it_counter}" + " -> ALPHA = " + str(alpha) + "\n")
    start = datetime.now()

    if it_counter == 1:
        first_half, chunks = split_dataset(NUM_ITERATIONS, 111)
        matrix = get_starting_dataset(first_half)
    else:
        matrix = enrich_dataset(matrix, chunks[it_counter-2])

    # Compute Similarities
    print("First Step: Compute Similarities of each User of the Group")
    list_of_similarities = list()
    for user in group:
        dictionary = dict()

        other_users = [u for u in matrix.index if u != user]
        for u in other_users:
            dictionary[u] = pearson_similarity(matrix, user, u)
        list_of_similarities.append(dictionary)
    
    # Neighborhood (75) Building
    print("Second Step: Neighborhood Building")
    for i in range(0, len(list_of_similarities)):
        similarities = list_of_similarities.pop(i)
        similarities = {k: v for k, v in sorted(similarities.items(), key=lambda item: item[1], reverse=True)}
        similarities = dict(list(similarities.items())[:MAX_NEIGHBORS])
        list_of_similarities.insert(i, similarities)
    
    # Predictions for Each User
    print("Third Step: Predictions for Each User")
    list_of_scores = list()

    for i in range(0, len(group)):
        scores = prediction_function(matrix, group[i], list_of_similarities[i], matrix.shape[1])
        list_of_scores.append(scores)
    
    # Group Recommendation
    print("Fourth Step: Group Recommendation")
    score_matrix = alpha_hybrid_aggregation(alpha, list_of_scores)

    # Evaluation: Group Satisfaction & Group Disagreements
    print("Fifth Step: Evaluation")
    for i, user in enumerate(group):
        sat = user_satisfaction(score_matrix, list_of_scores[i], MAX_RECOMMENDATIONS)
        df_user_sat.at[user, f"It{it_counter}"] = sat
        df_user_satO.at[user, f"It{it_counter}"] = np.mean(df_user_sat.loc[user])
        # print(f"User {user} -> Satisfaction: {sat:.5f}")
    
    group_satO = np.sum(df_user_satO[f'It{it_counter}'])/3
    print("Group Satisfaction Overall: " + str(group_satO))

    group_dis = np.max(df_user_satO[f'It{it_counter}']) - np.min(df_user_satO[f'It{it_counter}'])
    group_disagreements.append(group_dis)
    print("Group Disagreement Overall: " + str(group_dis))

    # Getting ready for the Next Iteration
    alpha = update_alpha_mom_disagreements(df_user_sat[f"It{it_counter}"])

    end = datetime.now()
    print(f"END OF ITERATION {it_counter} - Elapsed Time: {end-start}" + "\n")

    it_counter += 1

ITERATION: 1 -> ALPHA = 0

First Step: Compute Similarities of each User of the Group
Second Step: Neighborhood Building
Third Step: Predictions for Each User
Fourth Step: Group Recommendation
Fifth Step: Evaluation
Group Satisfaction Overall: 0.8075472651387199
Group Disagreement Overall: 0.1383570854280205
END OF ITERATION 1 - Elapsed Time: 0:00:33.848513

ITERATION: 2 -> ALPHA = 0.07944373636743063

First Step: Compute Similarities of each User of the Group
Second Step: Neighborhood Building
Third Step: Predictions for Each User
Fourth Step: Group Recommendation
Fifth Step: Evaluation
Group Satisfaction Overall: 0.8136062018467873
Group Disagreement Overall: 0.09458940281397665
END OF ITERATION 2 - Elapsed Time: 0:00:28.937051

ITERATION: 3 -> ALPHA = 0.025558138799979258

First Step: Compute Similarities of each User of the Group
Second Step: Neighborhood Building
Third Step: Predictions for Each User
Fourth Step: Group Recommendation
Fifth Step: Evaluation
Group Satisfaction Overa

# First Execution (seed == 24) Outcomes

In [45]:
df_user_sat

,It1,It2,It3,It4,It5
437,0.899719,0.900556,0.86303,0.884683,0.926059
329,0.802304,0.795401,0.720284,0.7753,0.731574
73,0.94698,0.89671,0.76643,0.719384,0.717571


In [46]:
# seed == 24
df_user_satO

,It1,It2,It3,It4,It5
437,0.899719,0.900138,0.887769,0.886997,0.894809
329,0.802304,0.798852,0.772663,0.773322,0.764972
73,0.94698,0.921845,0.87004,0.832376,0.809415


In [47]:
df_user_satO.mean()

It1    0.883001
It2    0.873612
It3     0.84349
It4    0.830898
It5    0.823066
dtype: object

In [48]:
group_disagreements

[0.14467521805143935,
 0.12299211255482911,
 0.11510559325028014,
 0.11367508444435981,
 0.1298370053856337]

# Second Execution (seed == 8) Outcomes

In [51]:
df_user_sat

,It1,It2,It3,It4,It5
437,0.89246,0.911853,0.917832,0.864552,0.926059
329,0.689805,0.682478,0.74612,0.76865,0.731574
73,0.86094,0.846825,0.751292,0.680177,0.717571


In [52]:
# seed == 8
df_user_satO

,It1,It2,It3,It4,It5
437,0.89246,0.902156,0.907381,0.896674,0.902551
329,0.689805,0.686142,0.706134,0.721763,0.723725
73,0.86094,0.853883,0.819686,0.784809,0.771361


In [53]:
df_user_satO.mean()

It1    0.814402
It2     0.81406
It3    0.811067
It4    0.801082
It5    0.799213
dtype: object

In [54]:
group_disagreements

[0.20265472327333545,
 0.21601468333249518,
 0.201247013150203,
 0.17491087063032607,
 0.17882563433440668]

# Third Execution (seed == 111) Outcomes

In [57]:
df_user_sat

,It1,It2,It3,It4,It5
437,0.892942,0.836704,0.878283,0.90004,0.926059
329,0.754585,0.785882,0.828792,0.763266,0.731574
73,0.775115,0.836409,0.789393,0.789563,0.717571


In [58]:
# seed == 111
df_user_satO

,It1,It2,It3,It4,It5
437,0.892942,0.864823,0.86931,0.876992,0.886806
329,0.754585,0.770233,0.789753,0.783131,0.77282
73,0.775115,0.805762,0.800306,0.79762,0.78161


In [59]:
df_user_satO.mean()

It1    0.807547
It2    0.813606
It3    0.819789
It4    0.819248
It5    0.813745
dtype: object

In [60]:
group_disagreements

[0.1383570854280205,
 0.09458940281397665,
 0.07955660462807046,
 0.09386101801830427,
 0.11398575224478924]